In [1]:
# the target is to gernerate a series of measurement script to qasm and
# then to analyse them.
# First function should be submission circuit
#from api_wrappers import HoneywellQAPI as QAPI
#import time
#import config
from scipy.optimize import minimize
import numpy as np
import qiskit as qk
import networkx as nx
import tenpy
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city
# custom things
from isonetwork import IsoTensor, IsoNetwork, QKParamCircuit
import mps
import networks
import random

##useless part for Daoheng:
def fh_bases_gen(s,t,l):
    bases = ''
    for i in range(l):
        if i>s-1 and i < t:
            bases += 'b'
        else: 
            bases += 'a'
    return bases

def mps_meausrement(bases, FH, shots, preg, breg, pcircs):
    start = time.time()
    # run job and retrieve data
    psi_mps = IsoMPS(preg,breg,pcircs,bases=bases,FH=FH)
    openqasm = psi_mps
    job_id = qapi.submit_job(openqasm, shots=shots, name='repetition code') 
    jr = qapi.retrieve_job(job_id)


'''
# run job and retrieve data
job_id = qapi.submit_job(openqasm, shots=shots, name='repetition code') 
jr = qapi.retrieve_job(job_id)
print('Total time = {0:1.4f} s'.format(time.time() - start))
print('\nRaw output:')
print(jr)'''

"\n# run job and retrieve data\njob_id = qapi.submit_job(openqasm, shots=shots, name='repetition code') \njr = qapi.retrieve_job(job_id)\nprint('Total time = {0:1.4f} s'.format(time.time() - start))\nprint('\nRaw output:')\nprint(jr)"

In [2]:
# heisenberg point, del = 1
def energy_estimator(val,shots,nvp):
#defining the circuit
    nphys = 1
    nbond = 1
    l_uc = 10
    energy = 0
    simulator = qk.providers.aer.QasmSimulator(method='matrix_product_state')
    preg = qk.QuantumRegister(nphys,'p')
    breg = qk.QuantumRegister(nbond,'b')
    qregs = [preg,breg]
    cregs = [qk.ClassicalRegister(nphys,'m'+str(j)) for j in range(l_uc)]
    # setup variational parameters; ******might need to change*****
    vals = val*np.ones(1)#list of pvalue
    print(vals)
    plabels = [qk.circuit.Parameter('x'+str(j))for j in range(nvp)]
    print(plabels[0])
    paramdict = dict(zip(plabels,vals))
    circs=[qk.QuantumCircuit() for j in range(l_uc)]

    #put the random number generator?
    
    # the varational circuit part, need to be changed to the model we work on, 
    # here is the heisenberg model at the antiferromagnetic Heisenberg poin
    for j in range(l_uc):
        for reg in qregs: circs[j].add_register(reg)
        if j%2 ==1:
            circs[j].x(preg[0])
        circs[j].rx(np.pi/2,breg[0])
        circs[j].ry(np.pi/2,preg[0])
        circs[j].cz(breg[0],preg[0])
        #parameter gate
        circs[j].rx(-1*plabels[0],breg[0])
        circs[j].ry(1*plabels[0],preg[0])
        circs[j].cz(breg[0],preg[0])
        circs[j].rx(-np.pi/2,breg[0])
        circs[j].ry(-np.pi/2,preg[0])
    pcircs = []
    for j in range(l_uc):
        pcircs += [QKParamCircuit(circs[j],[plabels[0]])]
        
#running the cirucuit
    #xx part ******might need to change***** 
    psi_mps = networks.IsoMPS(preg,breg,pcircs,bases=['zzzzzzzzxx'])
    circx = psi_mps.construct_circuit(paramdict)
    result = execute(circx, simulator, shots=shots, memory=True).result()
    
    countx = result.get_counts(circx)
    C_XX = np.zeros(l_uc) # XX correlator
    for k,v in countx.items():
        for x in range(l_uc):
            C_XX[x] += v * (1.0-2.0*np.float(k.replace(" ", "")[l_uc-x-1]=='1'))*(1.0-2.0*np.float(k.replace(" ", "")[l_uc-x-2]=='1'))/shots
    #zz part ******might need to change*****
    psi_mps = networks.IsoMPS(preg,breg,pcircs,bases=['zzzzzzzzzz'])
    circz = psi_mps.construct_circuit(paramdict)
    circz.draw()
    result = execute(circz, simulator, shots=shots, memory=True).result()
    countz = result.get_counts(circz)
    C_ZZ = np.zeros(l_uc) # ZZ correlator
    for k,v in countz.items():
        for x in range(l_uc):
            C_ZZ[x] += v * (1.0-2.0*np.float(k.replace(" ", "")[l_uc-x-1]=='1'))*(1.0-2.0*np.float(k.replace(" ", "")[l_uc-x-2]=='1'))/shots
    energy+= 2*C_XX[l_uc-2] + C_ZZ[l_uc-2]
    return energy
#p=energy_estimator(shots=5000,val=0.9,nvp=1)
result = minimize(energy_estimator, x0=1.18, args=(5000,1), method='BFGS', options={'gtol': 1e-7, 'disp': True, 'maxiter': 10})

[1.18]
x0
[1.18000001]
x0
[0.17]
x0
[0.17000001]
x0
[1.12146115]
x0
[1.12146117]
x0
[1.14765336]
x0
[1.14765338]
x0
[1.16157567]
x0
[1.16157569]
x0
[1.17007598]
x0
[1.17007599]
x0
[1.17448746]
x0
[1.17448748]
x0
[1.17683583]
x0
[1.17683585]
x0
[1.17916111]
x0
[1.17916112]
x0
[1.17952799]
x0
[1.17952801]
x0
[1.17974016]
x0
[1.17974018]
x0
[1.17973984]
x0
[1.17973985]
x0
[1.17974001]
x0
[1.17974002]
x0
[1.17974015]
x0
[1.17974017]
x0
[1.17974012]
x0
[1.17974014]
x0
[1.17974015]
x0
[1.17974016]
x0
[1.17974015]
x0
[1.17974017]
x0
[1.17974015]
x0
[1.17974017]
x0
[1.17974015]
x0
[1.17974017]
x0
[1.17974015]
x0
[1.17974017]
x0
[1.17974015]
x0
[1.17974017]
x0
[1.17973984]
x0
[1.17974001]
x0
[1.17974012]
x0
[1.17974015]
x0
[1.17974017]
x0
[1.17974013]
x0
[1.17974015]
x0
[1.17974016]
x0
[1.17974014]
x0
[1.17974015]
x0
[1.17974015]
x0
[1.17974015]
x0
[1.17974016]
x0
[1.17974015]
x0
[1.17974015]
x0
         Current function value: -1.720800
         Iterations: 1
         Function evaluations: 57


In [3]:
#sample circuit
cir =qk.QuantumCircuit(2,2)
cir.h(0)
cir.cz(0,1)
cir.h(1)
cir.measure([0,1],[0,1])
cir.draw()
simulator = QasmSimulator(method='matrix_product_state')
result = execute(cir, simulator, shots=2000, memory=True).result()

NameError: name 'QasmSimulator' is not defined

In [445]:
for a,b in result.get_counts().items(): print(a,b)

00 518
01 475
10 493
11 514
